In [1]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-
"""
Run a YOLO_v3 style detection model on test images.
"""
import colorsys
import os
import random
import numpy as np
import cv2

from timeit import time
from timeit import default_timer as timer  # to calculate FPS
from keras  import backend as K
from keras.models import load_model
from yolo3.model import yolo_eval

font = cv2.FONT_HERSHEY_SIMPLEX

class YOLO(object):
    def __init__(self):
        self.model_path   = 'model_data/yolo.h5'
        self.anchors_path = 'model_data/yolo_anchors.txt'
        self.classes_path = 'model_data/coco_classes.txt'
        self.score = 0.3
        self.iou = 0.5
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.model_image_size = (416, 416)  # fixed size or (None, None)
        self.is_fixed_size = self.model_image_size != (None, None)
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
            anchors = [float(x) for x in anchors.split(',')]
            anchors = np.array(anchors).reshape(-1, 2)
        return anchors

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model must be a .h5 file.'

        self.yolo_model = load_model(model_path, compile=False)
        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        random.seed(10101)  # Fixed seed for consistent colors across runs.
        # Shuffle colors to decorrelate adjacent classes.
        random.shuffle(self.colors)
        random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                                           len(self.class_names), self.input_image_shape,
                                           score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_data(self, image, animals, predictEdge, pos, database):
        predictEdge /= 100
        if self.is_fixed_size:
            assert self.model_image_size[0] % 32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1] % 32 == 0, 'Multiples of 32 required'
            boxed_image = cv2.resize(image,(416,416))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = cv2.resize(image,(416,416))
        image_data = np.array(boxed_image, dtype='float32')
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        thickness = (416 + 416) // 300

        for unit in database.find({"pos": pos}):
            c = unit["predict"]
            predicted_class = self.class_names[c]
            score = unit["score"]
            # print(c, score, predictEdge)
            if((score < predictEdge) or ((animals >> c) & 1 == 0)):
                continue

            label = '{} {:.2f}'.format(predicted_class, score)
            label_size = 10

            top = unit["top"]
            left = unit["left"]
            bottom = unit["bottom"]
            right = unit["right"]

            if top - label_size >= 0:
                text_origin = (left, top - label_size)
            else:
                text_origin = (left, top + 1)
            for i in range(thickness):
                im.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=self.colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=self.colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw
        return image

    def detect_image(self, image, animals=1, predictEdge=0, pos=-1, database=None):
        predictEdge /= 100
        if self.is_fixed_size:
            assert self.model_image_size[0] % 32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1] % 32 == 0, 'Multiples of 32 required'
            boxed_image = cv2.resize(image,(416,416))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = cv2.resize(image,(416,416))
        image_data = np.array(boxed_image, dtype='float32')

        # print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

	#Feeding The Network.
        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [416, 416],
                K.learning_phase(): 0
            })

        # print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        thickness = (416 + 416) // 300

        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]
            score = float(score)
            score = round(score, 2)
            # print(score, predicted_class, c)
            if((score < predictEdge) or ((animals >> c) & 1 == 0)):
                continue
            label = '{} {:.2f}'.format(predicted_class, score)
            label_size = 10

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(416, np.floor(bottom + 0.5).astype('int32'))
            right =  min(416, np.floor(right + 0.5).astype('int32'))
            # print(label, (left, top), (right, bottom), pos, c, type(top))
            if(pos != -1):
                database.insert({"pos": pos, "left": int(left), "right": int(right),
                                 "top": int(top), "bottom": int(bottom), "predict": int(c), "score": score})
            if top - label_size >= 0:
                text_origin = np.array([left, top - label_size])
            else:
                text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                cv2.rectangle(image,(left + i,top + i),(right - i,bottom - i)
                              ,self.colors[c],3)
            cv2.putText(image,label,tuple(text_origin), font, 1,(255,255,255),2)
        # end = time.time()
        # print(end - start)
        return image

    def close_session(self):
        self.sess.close()


def detect_video(yolo, video_path=0):
    import cv2
    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    cv2.namedWindow("result", cv2.WINDOW_NORMAL)
    while True:
        return_value, frame = vid.read()
        image = Image.fromarray(frame)
        image = yolo.detect_image(image)
        result = np.asarray(image)
        curr_time = timer()
        exec_time = curr_time - prev_time
        prev_time = curr_time
        accum_time = accum_time + exec_time
        curr_fps = curr_fps + 1
        if accum_time > 1:
            accum_time = accum_time - 1
            fps = "FPS: " + str(curr_fps)
            curr_fps = 0
        cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=2)

        cv2.imshow("result", result)
        tt = cv2.waitKey(1)
        print("hh %d" % (tt))
        if tt == 113:
            break
    cv2.destroyAllWindows()
    # yolo.close_session()


def detect_img(yolo):
    image = cv2.imread('test.jpg',cv2.IMREAD_COLOR)
    r_image = yolo.detect_image(image)
    cv2.imshow('image',image)
    cv2.waitKey(0)   
    cv2.destroyAllWindows()
    yolo.close_session()

Using TensorFlow backend.


In [ ]:
detect_img(YOLO())

model_data/yolo.h5 model, anchors, and classes loaded.
